# Примечание 
У меня получилось далеко не все, надеюсь разобраться спустя некоторое время 

# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI.

In [1]:
%pip install wikipedia

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     -- ------------------------------------- 10.2/143.0 kB ? eta -:--:--
     --------------- --------------------- 61.4/143.0 kB 465.5 kB/s eta 0:00:01
     --------------------------------- -- 133.1/143.0 kB 653.6 kB/s eta 0:00:01
     ------------------------------------ 143.0/143.0 kB 653.9 kB/s eta 0:00:00
  Obtaining dependency information for soupsieve>1.2 from https://files.pythonhosted.org/packages/4c/f3/038b302fdfbe3be7da016777069f26ceefe11a681055ea1f7817546508e3/soupsieve-2.5-py3-none-any.whl.metadata
  Cre


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import wikipedia
import numpy as np
from collections import defaultdict
from string import digits
import re
import nltk.data
from nltk.tokenize import sent_tokenize
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import adjusted_rand_score
from sentence_transformers import SentenceTransformer


c:\Users\Home\scoop\apps\python\current\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
code2lang = wikipedia.languages()

In [15]:
list(code2lang.items())[11:500]

[('aln', 'Gegë'),
 ('als', 'Alemannisch'),
 ('alt', 'алтай тил'),
 ('am', 'አማርኛ'),
 ('ami', 'Pangcah'),
 ('an', 'aragonés'),
 ('ang', 'Ænglisc'),
 ('ann', 'Obolo'),
 ('anp', 'अंगिका'),
 ('ar', 'العربية'),
 ('arc', 'ܐܪܡܝܐ'),
 ('arn', 'mapudungun'),
 ('arq', 'جازايرية'),
 ('ary', 'الدارجة'),
 ('arz', 'مصرى'),
 ('as', 'অসমীয়া'),
 ('ase', 'American sign language'),
 ('ast', 'asturianu'),
 ('atj', 'Atikamekw'),
 ('av', 'авар'),
 ('avk', 'Kotava'),
 ('awa', 'अवधी'),
 ('ay', 'Aymar aru'),
 ('az', 'azərbaycanca'),
 ('azb', 'تۆرکجه'),
 ('ba', 'башҡортса'),
 ('ban', 'Basa Bali'),
 ('ban-bali', 'ᬩᬲᬩᬮᬶ'),
 ('bar', 'Boarisch'),
 ('bat-smg', 'žemaitėška'),
 ('bbc', 'Batak Toba'),
 ('bbc-latn', 'Batak Toba'),
 ('bcc', 'جهلسری بلوچی'),
 ('bci', 'wawle'),
 ('bcl', 'Bikol Central'),
 ('bdr', 'Bajau Sama'),
 ('be', 'беларуская'),
 ('be-tarask', 'беларуская (тарашкевіца)'),
 ('be-x-old', 'беларуская (тарашкевіца)'),
 ('bew', 'Betawi'),
 ('bg', 'български'),
 ('bgn', 'روچ کپتین بلوچی'),
 ('bh', 'भोजपुरी')

In [16]:

langs = ['uz', 'ce', 'tt', 'it', 'is']
[(lang, code2lang[lang]) for lang  in langs]

[('uz', 'oʻzbekcha / ўзбекча'),
 ('ce', 'нохчийн'),
 ('tt', 'татарча / tatarça'),
 ('it', 'italiano'),
 ('is', 'íslenska')]

In [11]:
def load_with_disambigution(page):
    try:
        p = wikipedia.page(page)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        p = wikipedia.page(random_option)
    return p

In [12]:
def get_texts_for_lang(lang, n=100): # функция для скачивания статей из википедии
    # вот так можно зафиксировать язык
    wikipedia.set_lang(lang)
    wiki_content = []
    
    # random достает только ссылки на статьи
    pages = wikipedia.random(n)
    
    for page_name in pages:
        try:
            # чтобы загрузить контент статьи можно вызвать функцию .page
            page = load_with_disambigution(page_name)
        
        except Exception as e:
            print('Skipping page {}'.format(page_name), e)
            continue

        wiki_content.append(f'{page.title}\n{page.content.replace("==", "")}')

    return wiki_content

In [131]:
def remove_irr_char_func(text):
    stroka = re.sub('\\\\n', ' ', text)
    stroka = re.sub('\n', ' ', stroka)
    stroka = stroka.replace('\\', '')
    stroka = stroka.replace('—', '')
    stroka = stroka.replace('/', '')
    stroka = re.sub(r"[()]", " ", stroka)
    stroka = re.sub(r'^\s*|\s\s*', ' ', stroka).strip()
    return stroka

In [21]:
wiki_texts = {}

for lang in langs:
    try:
        wiki_texts[lang] = get_texts_for_lang(lang, 10)
    except Exception as e:
        print('ERROR ON - ', lang, e)
        continue
    
    print(lang, len(wiki_texts[lang]))

uz 10
ce 10
tt 10
it 10
is 10


In [165]:
clean_text_ce = remove_irr_char_func(str(wt['ce']))
sentences_ce = sent_tokenize(clean_text_ce)
ce = list()
for x in range(len(sentences_ce)):
    ce.append('ce')
    
df_ce = pd.DataFrame(list(zip(ce, sentences_ce)), columns = ['language', 'sent'])


In [153]:
clean_text_uz = remove_irr_char_func(str(wt['uz']))
sentences_uz = sent_tokenize(clean_text_uz)
uz = list()
for x in range(len(sentences_uz)):
    uz.append('uz')
    
df_uz = pd.DataFrame(list(zip(uz, sentences_uz)), columns = ['language', 'sent'])

In [156]:
clean_text_tt = remove_irr_char_func(str(wt['tt']))
sentences_tt = sent_tokenize(clean_text_tt)
tt = list()
for x in range(len(sentences_tt)):
    tt.append('tt')
    
df_tt = pd.DataFrame(list(zip(tt, sentences_tt)), columns = ['language', 'sent'])

In [158]:
clean_text_it = remove_irr_char_func(str(wt['it']))
sentences_it = sent_tokenize(clean_text_it)
it = list()
for x in range(len(sentences_it)):
    it.append('it')
    
df_it = pd.DataFrame(list(zip(it, sentences_it)), columns = ['language', 'sent'])

In [199]:
clean_text_is = remove_irr_char_func(str(wt['is']))
sentences_is = sent_tokenize(clean_text_is)
island = list()
for x in range(len(sentences_is)):
    island.append('is')
    
df_is = pd.DataFrame(list(zip(island, sentences_is)), columns = ['language', 'sent'])

In [206]:
extra = []
for x in range(len(dataframe)):
    extra.append('extra')

In [190]:
dataframe = pd.concat([df_ce,df_uz,df_tt,df_it,df_is])

In [207]:
dataframe['extra'] = extra

In [208]:
dataframe

,language,sent,extra
0,ce,['Гугаль Гугаль серб.,extra
1,ce,Гугаљ Сербин Златибор гуонан Пожега йукъаралле...,extra
2,ce,општина нах беха меттиг.,extra
3,ce,Географи Лаьтташ йу Балканийн ахгӀайренан къил...,extra
4,ce,"Климат Кхузахь климат барамехь континентан ю, ...",extra
...,...,...,...
165,is,KSÍ.,extra
166,is,Sótt 27. september 2023.,extra
167,is,"', 'Hampden Park Hampden Park er knattspyrnuvö...",extra
168,is,Hann er heimavöllur Skoska landsliðsins og hýs...,extra


In [209]:
grouped_df = dataframe.groupby('extra')[['language', 'sent', 'extra']]

In [210]:
for key, _ in grouped_df:
    print(grouped_df.get_group(key), "\n\n")
    break

    language                                               sent  extra
0         ce                              ['Гугаль Гугаль серб.  extra
1         ce  Гугаљ Сербин Златибор гуонан Пожега йукъаралле...  extra
2         ce                           општина нах беха меттиг.  extra
3         ce  Географи Лаьтташ йу Балканийн ахгӀайренан къил...  extra
4         ce  Климат Кхузахь климат барамехь континентан ю, ...  extra
..       ...                                                ...    ...
165       is                                               KSÍ.  extra
166       is                           Sótt 27. september 2023.  extra
167       is  ', 'Hampden Park Hampden Park er knattspyrnuvö...  extra
168       is  Hann er heimavöllur Skoska landsliðsins og hýs...  extra
169       is           Völlurinn tekur rúmlega 50.000 í sæti.']  extra

[1054 rows x 3 columns] 




In [211]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

In [214]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['sent'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(3)
#     cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['language'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Home\scoop\apps\python\current\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.5592790868130846


# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

In [ ]:
!wget https://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2

In [1]:
import bz2

with bz2.open('annot.opcorpora.no_ambig_strict.xml.bz2', 'rb') as f_in, open('annot.opcorpora.no_ambig_strict.xml', 'wb') as f_out:
    f_out.write(f_in.read())

In [2]:
from lxml import etree
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import classification_report
import numpy as np
from collections import Counter

In [3]:
import spacy_udpipe

In [4]:
open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

In [5]:
sentences = open_corpora.xpath('//tokens')
tokens = sentences[1].xpath('token')
[token.xpath('@text')[0] for token in tokens]

['Сохранится', 'ли', 'градус', 'дискуссии', 'в', 'новом', 'сезоне', '?']

## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуру через DependencyGraph, выберите отношение (из тех, что не упоминались на семинаре) и найдите самые частотные пары слов, связанных этим отношением. 

Для самой частотной пары вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)`) В итоге у вас должен получится список пары: слово1 и все его подзависимые - слово2 и все его подзависимые.

Визуализируйте самое длинное предложение через networkx

In [1]:
from nltk.grammar import DependencyGrammar
from nltk.parse import DependencyGraph
from collections import Counter
import numpy as np
import re
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
file = open('ru_syntagrus-ud-test.conllu', encoding='utf-8').read()

In [3]:
tree = [x for x in file.split('\n\n')[0].split('\n')]

In [4]:
tree

['# sent_id = 2003Armeniya.xml_1',
 '# text = В советский период времени число ИТ- специалистов в Армении составляло около десяти тысяч.',
 '1\tВ\tв\tADP\t_\t_\t3\tcase\t3:case\t_',
 '2\tсоветский\tсоветский\tADJ\t_\tAnimacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing\t3\tamod\t3:amod\t_',
 '3\tпериод\tпериод\tNOUN\t_\tAnimacy=Inan|Case=Acc|Gender=Masc|Number=Sing\t11\tobl\t11:obl:в:acc\t_',
 '4\tвремени\tвремя\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Neut|Number=Sing\t3\tnmod\t3:nmod:gen\t_',
 '5\tчисло\tчисло\tNOUN\t_\tAnimacy=Inan|Case=Acc|Gender=Neut|Number=Sing\t11\tobj\t11:obj\t_',
 '6\tИТ\tИТ\tPROPN\t_\tAnimacy=Inan|Case=Nom|Gender=Neut|Number=Sing\t8\tcompound\t8:compound\tSpaceAfter=No',
 '7\t-\t-\tPUNCT\t_\t_\t6\tpunct\t6:punct\t_',
 '8\tспециалистов\tспециалист\tNOUN\t_\tAnimacy=Anim|Case=Gen|Gender=Masc|Number=Plur\t5\tnmod\t5:nmod:gen\t_',
 '9\tв\tв\tADP\t_\t_\t10\tcase\t10:case\t_',
 '10\tАрмении\tАрмения\tPROPN\t_\tAnimacy=Inan|Case=Loc|Gender=Fem|Number=Sing\t5\tnmod

In [44]:
trees = []

parsed_sents = open('ru_syntagrus-ud-test.conllu', encoding='utf-8').read().split('\n\n')
for sent in parsed_sents:
    tree = [line for line in sent.split('\n') if line[0] != '#']
    trees.append('\n'.join(tree))

In [35]:
[x.replace('\t', ' ') for x in trees[1].split('\n')]

['1 Доставшийся достаться VERB _ Aspect=Perf|Case=Nom|Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part|Voice=Mid 11 acl 11:acl _',
 '2 в в ADP _ _ 3 case 3:case _',
 '3 наследство наследство NOUN _ Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing 1 obl 1:obl:в:acc _',
 '4 от от ADP _ _ 6 case 6:case _',
 '5 советского советский ADJ _ Case=Gen|Degree=Pos|Gender=Masc|Number=Sing 6 amod 6:amod _',
 '6 периода период NOUN _ Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing 3 nmod 3:nmod:от:gen _',
 '7 времени время NOUN _ Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing 6 nmod 6:nmod:gen _',
 '8 промышленный промышленный ADJ _ Case=Nom|Degree=Pos|Gender=Masc|Number=Sing 11 amod 11:amod _',
 '9 и и CCONJ _ _ 10 cc 10:cc _',
 '10 интеллектуальный интеллектуальный ADJ _ Case=Nom|Degree=Pos|Gender=Masc|Number=Sing 8 conj 8:conj|11:amod _',
 '11 потенциал потенциал NOUN _ Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing 12 nsubj 12:nsubj _',
 '12 оказался оказаться VERB _ Aspect=Perf|Gender=Masc|Mood=Ind|Nu

In [38]:
d = DependencyGraph(trees[1])

In [39]:
d.nodes

defaultdict(<function nltk.parse.dependencygraph.DependencyGraph.__init__.<locals>.<lambda>()>,
            {0: {'address': 0,
              'word': None,
              'lemma': None,
              'ctag': 'TOP',
              'tag': 'TOP',
              'feats': None,
              'head': None,
              'deps': defaultdict(list, {'root': [12], 'ROOT': []}),
              'rel': None},
             1: {'address': 1,
              'word': 'Доставшийся',
              'lemma': 'достаться',
              'ctag': 'VERB',
              'tag': '_',
              'feats': 'Aspect=Perf|Case=Nom|Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part|Voice=Mid',
              'head': 11,
              'deps': defaultdict(list, {'obl': [3]}),
              'rel': 'acl'},
             11: {'address': 11,
              'word': 'потенциал',
              'lemma': 'потенциал',
              'ctag': 'NOUN',
              'tag': '_',
              'feats': 'Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing

In [49]:
d.root = d.nodes[0] # чтобы метод сработал нужно установить корневой элемент
triples = list(d.triples())

modifiers = Counter()
for e1, rel, e2 in triples:
    if e1[0] == 'корпусом' and rel == 'amod':
        modifiers[e2[0]] += 1

In [48]:
triples

[((None, 'TOP'), 'root', ('покинул', 'VERB')),
 (('покинул', 'VERB'), 'obl', ('началом', 'NOUN')),
 (('началом', 'NOUN'), 'case', ('С', 'ADP')),
 (('началом', 'NOUN'), 'nmod', ('Европе', 'PROPN')),
 (('Европе', 'PROPN'), 'case', ('в', 'ADP')),
 (('началом', 'NOUN'), 'nmod', ('войны', 'NOUN')),
 (('войны', 'NOUN'), 'nmod', ('Наполеоном', 'PROPN')),
 (('Наполеоном', 'PROPN'), 'case', ('с', 'ADP')),
 (('покинул', 'VERB'), 'nsubj', ('Платов', 'PROPN')),
 (('покинул', 'VERB'), 'advmod', ('вместе', 'ADV')),
 (('вместе', 'ADV'), 'obl', ('корпусом', 'NOUN')),
 (('корпусом', 'NOUN'), 'case', ('с', 'ADP')),
 (('корпусом', 'NOUN'), 'amod', ('казачьим', 'ADJ')),
 (('покинул', 'VERB'), 'obj', ('Дон', 'PROPN')),
 (('Дон', 'PROPN'), 'amod', ('родной', 'ADJ')),
 (('покинул', 'VERB'), 'conj', ('отправился', 'VERB')),
 (('отправился', 'VERB'), 'cc', ('и', 'CCONJ')),
 (('отправился', 'VERB'), 'obl', ('расположение', 'NOUN')),
 (('расположение', 'NOUN'), 'case', ('в', 'ADP')),
 (('расположение', 'NOUN'), 

In [50]:
modifiers

Counter({'казачьим': 1})

In [105]:
for tree in trees[:726]:
    d = DependencyGraph(tree)

AssertionError: 

In [103]:
modifiers = Counter()
for tree in trees[:725]:
    d = DependencyGraph(tree)
    d.root = d.nodes[0] 
    triples = list(d.triples())
    
    for e1, rel, e2 in triples:
        if e1[0] == 'войск' and rel == 'amod':
            modifiers[e2[0]] += 1
    # ваш код здесь
    

In [104]:
modifiers

Counter({'пугачевских': 1, 'казачьих': 1, 'русских': 1})